In [1]:
import sys
import os
os.chdir("../")

In [16]:
import pandas as pd
import numpy as np
import data
from tqdm.auto import tqdm
from preprocess_utils.last_clickout_indices import find as find_last_clickout_indices
from collections import Counter
from sklearn.preprocessing import MinMaxScaler

tqdm.pandas()
from IPython.display import display
pd.options.display.max_columns = None

In [3]:
df = data.train_df('small')

In [6]:
df

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices,frequence
0,00RL8Z82B2Z1,aff3928535f48,1541037460,1,search for poi,Newtown,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1.0
1,00RL8Z82B2Z1,aff3928535f48,1541037522,2,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,3.0
4,00RL8Z82B2Z1,aff3928535f48,1541037532,5,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1.0
5,00RL8Z82B2Z1,aff3928535f48,1541037532,6,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1.0
6,00RL8Z82B2Z1,aff3928535f48,1541037532,7,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1.0
7,00RL8Z82B2Z1,aff3928535f48,1541037532,8,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1.0
8,00RL8Z82B2Z1,aff3928535f48,1541037542,9,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,5.0
13,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,NaN,3400638|1253714|3367857|5100540|1088584|666916...,95|66|501|112|95|100|101|72|82|56|56|143|70|25...,1.0
14,00RL8Z82B2Z1,aff3928535f48,1541038469,15,search for poi,Surry Hills,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1.0
15,00RL8Z82B2Z1,aff3928535f48,1541038485,16,clickout item,1257342,AU,"Sydney, Australia",mobile,NaN,55109|129343|54824|2297972|109014|1257342|1031...,162|25|150|143|101|49|118|131|18|100|101|143|5...,1.0


Count items popularity in full!

In [4]:
cnt = Counter(df[(df.action_type == 'clickout item') & (df.reference.str.isnumeric() == True)].reference.values.astype(int))

In [8]:
cnt.most_common(10)

[(8796, 936),
 (8561, 872),
 (8621, 832),
 (8589, 804),
 (8586, 801),
 (8618, 725),
 (1455251, 715),
 (8805412, 641),
 (9773310, 639),
 (8588, 637)]

In [9]:
cnt[8796]

936

In [5]:
clickout_rows = df[df.action_type == 'clickout item'][['reference','impressions']]
clickout_rows = clickout_rows.fillna(-1).astype({'reference':'int'})
clickout_rows['impressions'] = clickout_rows.apply(lambda x: list(map(int,x.impressions.split('|'))), axis=1)

In [6]:
clickout_rows

,reference,impressions
13,109038,"[3400638, 1253714, 3367857, 5100540, 1088584, ..."
15,1257342,"[55109, 129343, 54824, 2297972, 109014, 125734..."
115,2795374,"[2795374, 5582964, 1088390, 2781070, 1258068, ..."
121,1032816,"[12693, 46363, 81657, 18448, 47687, 152913, 18..."
122,1032816,"[12693, 46363, 81657, 18448, 47687, 152913, 18..."
176,65685,"[1306936, 56482, 2842358, 6881276, 65685, 6325..."
177,1320460,"[1306936, 56482, 2842358, 6881276, 65685, 6325..."
180,3143258,"[1258184, 3866722, 8929970, 2315702, 116619, 1..."
181,2552514,"[2349076, 2552514, 7159866, 3898458, 1846017, ..."
184,110591,"[6721, 6724, 40109, 147227, 80983, 6719, 40718..."


In [7]:
matrix = np.zeros((clickout_rows.shape[0],25), dtype=int)

In [9]:
i = 0
for impr in tqdm(clickout_rows.impressions):
    for j,impr in enumerate(impr):
        #popularity = cnt[impr] if impr in cnt else 0
        #if popularity == row.reference:
        #    popularity -= 1
        popularity = cnt[impr]-1 if impr in cnt else 0
        matrix[i, j] = popularity
    i += 1

In [15]:
matrix

array([[2, 0, 0, ..., 0, 1, 0],
       [0, 0, 2, ..., 0, 0, 0],
       [2, 0, 1, ..., 0, 3, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [14]:
np.log(matrix+1)

array([[1.09861229, 0.        , 0.        , ..., 0.        , 0.69314718,
        0.        ],
       [0.        , 0.        , 1.09861229, ..., 0.        , 0.        ,
        0.        ],
       [1.09861229, 0.        , 0.69314718, ..., 0.        , 1.38629436,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [21]:
scaler = MinMaxScaler()
scaler.fit_transform(np.log(matrix+1))

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0.])

In [37]:
for i in range(25):
    clickout_rows['impr_pop{}'.format(i)] = matrix[:, i]

In [38]:
clickout_rows

,reference,impressions,impr_pop_0,impr_pop_1,impr_pop_2,impr_pop_3,impr_pop_4,impr_pop_5,impr_pop_6,impr_pop_7,impr_pop_8,impr_pop_9,impr_pop_10,impr_pop_11,impr_pop_12,impr_pop_13,impr_pop_14,impr_pop_15,impr_pop_16,impr_pop_17,impr_pop_18,impr_pop_19,impr_pop_20,impr_pop_21,impr_pop_22,impr_pop_23,impr_pop_24
13,109038,"[3400638, 1253714, 3367857, 5100540, 1088584, ...",3,0,1,0,1,0,1,0,0,5,1,1,0,1,0,0,1,2,0,0,0,1,0,2,0
15,1257342,"[55109, 129343, 54824, 2297972, 109014, 125734...",1,1,3,0,2,2,0,1,0,0,1,0,0,1,0,2,0,0,0,0,1,0,0,0,0
115,2795374,"[2795374, 5582964, 1088390, 2781070, 1258068, ...",3,1,2,0,2,2,0,3,3,2,0,0,0,0,0,0,1,2,0,0,0,1,0,4,0
121,1032816,"[12693, 46363, 81657, 18448, 47687, 152913, 18...",9,2,2,0,0,0,1,0,0,1,0,1,0,3,0,2,0,0,0,5,2,1,1,2,0
122,1032816,"[12693, 46363, 81657, 18448, 47687, 152913, 18...",9,2,2,0,0,0,1,0,0,1,0,1,0,3,0,2,0,0,0,5,2,1,1,2,0
176,65685,"[1306936, 56482, 2842358, 6881276, 65685, 6325...",0,2,0,0,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
177,1320460,"[1306936, 56482, 2842358, 6881276, 65685, 6325...",0,2,0,0,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
180,3143258,"[1258184, 3866722, 8929970, 2315702, 116619, 1...",0,0,0,0,2,1,1,0,0,0,0,2,0,6,0,0,0,0,0,3,0,1,2,2,0
181,2552514,"[2349076, 2552514, 7159866, 3898458, 1846017, ...",2,6,1,0,4,0,0,0,2,4,2,0,0,0,0,0,1,0,0,0,0,0,0,0,0
184,110591,"[6721, 6724, 40109, 147227, 80983, 6719, 40718...",1,4,3,1,1,1,0,1,0,0,0,8,0,2,1,0,0,0,1,1,0,0,0,3,0


In [4]:
from extract_features.rnn.impressions_popularity import ImpressionsPopularity
f = ImpressionsPopularity()

In [5]:
f.join_to(df)

/Users/federico/miniconda3/envs/recsys/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


impressions_popularity feature read


,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices,frequence,impr_pop0,impr_pop1,impr_pop2,impr_pop3,impr_pop4,impr_pop5,impr_pop6,impr_pop7,impr_pop8,impr_pop9,impr_pop10,impr_pop11,impr_pop12,impr_pop13,impr_pop14,impr_pop15,impr_pop16,impr_pop17,impr_pop18,impr_pop19,impr_pop20,impr_pop21,impr_pop22,impr_pop23,impr_pop24
0,00RL8Z82B2Z1,aff3928535f48,1541037460,1,search for poi,Newtown,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,00RL8Z82B2Z1,aff3928535f48,1541037522,2,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,3.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,00RL8Z82B2Z1,aff3928535f48,1541037532,5,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,00RL8Z82B2Z1,aff3928535f48,1541037532,6,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,00RL8Z82B2Z1,aff3928535f48,1541037532,7,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,00RL8Z82B2Z1,aff3928535f48,1541037532,8,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,00RL8Z82B2Z1,aff3928535f48,1541037542,9,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,5.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
13,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,NaN,3400638|1253714|3367857|5100540|1088584|666916...,95|66|501|112|95|100|101|72|82|56|56|143|70|25...,1.0,21,3,19,20,4,2,8,6,13,59,5,20,14,10,4,9,20,13,0,2,1,22,54,21,3
14,00RL8Z82B2Z1,aff3928535f48,1541038469,15,search for poi,Surry Hills,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
15,00RL8Z82B2Z1,aff3928535f48,1541038485,16,clickout item,1257342,AU,"Sydney, Australia",mobile,NaN,55109|129343|54824|2297972|109014|1257342|1031...,162|25|150|143|101|49|118|131|18|100|101|143|5...,1.0,35,10,87,3,28,21,10,22,9,2,8,54,19,20,19,16,5,0,4,4,4,13,9,1,2


In [6]:
_.dtypes

user_id            object
session_id         object
timestamp           int64
step                int64
action_type        object
reference          object
platform           object
city               object
device             object
current_filters    object
impressions        object
prices             object
frequence           int64
impr_c0              int8
impr_c1              int8
impr_c2              int8
impr_c3              int8
impr_c4              int8
impr_c5              int8
impr_c6              int8
impr_c7              int8
impr_c8              int8
impr_c9              int8
impr_c10             int8
impr_c11             int8
impr_c12             int8
impr_c13             int8
impr_c14             int8
impr_c15             int8
impr_c16             int8
impr_c17             int8
impr_c18             int8
impr_c19             int8
impr_c20             int8
impr_c21             int8
impr_c22             int8
impr_c23             int8
impr_c24             int8
dtype: objec